In [1]:
from flickrapi import FlickrAPI
import pandas as pd

In [2]:
key = '0e4e7a8ceb3f350abd0a743812365c51'
secret = '3172a3f6ec67c50f'
sizes = ['url_c', 'url_z', 'url_m', 'url_n', 'url_s', 'url_t']

In [3]:
def get_photos(image_tag):
    extras = ','.join(sizes)
    flickr = FlickrAPI(key, secret)
    photos = flickr.walk(text=image_tag,  # it will search by image title and image tags
                            extras=extras,  # get the urls for each size we want
                            privacy_filter=1,  # search only for public photos
                            per_page=50,
                            sort='relevance')  # we want what we are looking for to appear first
    return photos

In [30]:
get_photos('pulpo a la gallega')

<generator object FlickrAPI.data_walker at 0x7f1340a981b0>

In [5]:
def get_url(photo):
    for i in range(len(sizes)):  # makes sure the loop is done in the order we want
        url = photo.get(sizes[i])
        #print('Encontrado el tamaño ',sizes[i])
        if url:  # if url is None try with the next size
            print('Encontrado el tamaño ',sizes[i])
            return url

In [8]:
def get_urls(image_tag, max):
    
    photos = get_photos(image_tag)
    counter=0
    urls=[]

    for photo in photos:
        if counter < max:
            counter += 1
            print("Fetching url for image number {}".format(counter))
            try:
                url = get_url(photo)  # get preffered size url
                urls.append(url)
            except:    
                 print("Url for image number {} could not be fetched".format(count))
            # if no url for the desired sizes then try with the next photo
        else:
            print("Done fetching urls, fetched {} urls out of {}".format(len(urls), max))
            break
    print("Writing out the urls in the current directory")
    urls = pd.Series(urls)
    urls.to_csv(image_tag + '_urls.csv')
    print('Listo!')
    

    return urls

In [50]:
urls = get_urls('pulpo a la gallega', 1000)

Fetching url for image number 1
Encontrado el tamaño  url_c
Fetching url for image number 2
Encontrado el tamaño  url_c
Fetching url for image number 3
Encontrado el tamaño  url_c
Fetching url for image number 4
Encontrado el tamaño  url_c
Fetching url for image number 5
Encontrado el tamaño  url_c
Fetching url for image number 6
Encontrado el tamaño  url_m
Fetching url for image number 7
Encontrado el tamaño  url_c
Fetching url for image number 8
Encontrado el tamaño  url_c
Fetching url for image number 9
Encontrado el tamaño  url_c
Fetching url for image number 10
Encontrado el tamaño  url_c
Fetching url for image number 11
Encontrado el tamaño  url_c
Fetching url for image number 12
Encontrado el tamaño  url_c
Fetching url for image number 13
Encontrado el tamaño  url_c
Fetching url for image number 14
Encontrado el tamaño  url_c
Fetching url for image number 15
Encontrado el tamaño  url_c
Fetching url for image number 16
Encontrado el tamaño  url_c
Fetching url for image number 17


Fetching url for image number 151
Encontrado el tamaño  url_c
Fetching url for image number 152
Encontrado el tamaño  url_c
Fetching url for image number 153
Encontrado el tamaño  url_c
Fetching url for image number 154
Encontrado el tamaño  url_c
Fetching url for image number 155
Encontrado el tamaño  url_c
Fetching url for image number 156
Encontrado el tamaño  url_z
Fetching url for image number 157
Encontrado el tamaño  url_z
Fetching url for image number 158
Encontrado el tamaño  url_c
Fetching url for image number 159
Encontrado el tamaño  url_c
Fetching url for image number 160
Encontrado el tamaño  url_c
Fetching url for image number 161
Encontrado el tamaño  url_c
Fetching url for image number 162
Encontrado el tamaño  url_c
Fetching url for image number 163
Encontrado el tamaño  url_z
Fetching url for image number 164
Encontrado el tamaño  url_c
Fetching url for image number 165
Encontrado el tamaño  url_c
Fetching url for image number 166
Encontrado el tamaño  url_z
Fetching

Fetching url for image number 301
Encontrado el tamaño  url_c
Fetching url for image number 302
Encontrado el tamaño  url_c
Fetching url for image number 303
Encontrado el tamaño  url_c
Fetching url for image number 304
Encontrado el tamaño  url_c
Fetching url for image number 305
Encontrado el tamaño  url_c
Fetching url for image number 306
Encontrado el tamaño  url_c
Fetching url for image number 307
Encontrado el tamaño  url_c
Fetching url for image number 308
Encontrado el tamaño  url_c
Fetching url for image number 309
Encontrado el tamaño  url_c
Fetching url for image number 310
Encontrado el tamaño  url_c
Fetching url for image number 311
Encontrado el tamaño  url_z
Fetching url for image number 312
Encontrado el tamaño  url_c
Fetching url for image number 313
Encontrado el tamaño  url_z
Fetching url for image number 314
Encontrado el tamaño  url_c
Fetching url for image number 315
Encontrado el tamaño  url_z
Fetching url for image number 316
Encontrado el tamaño  url_c
Fetching

Fetching url for image number 451
Encontrado el tamaño  url_c
Fetching url for image number 452
Encontrado el tamaño  url_z
Fetching url for image number 453
Encontrado el tamaño  url_c
Fetching url for image number 454
Encontrado el tamaño  url_c
Fetching url for image number 455
Encontrado el tamaño  url_c
Fetching url for image number 456
Encontrado el tamaño  url_c
Fetching url for image number 457
Encontrado el tamaño  url_c
Fetching url for image number 458
Encontrado el tamaño  url_c
Fetching url for image number 459
Encontrado el tamaño  url_c
Fetching url for image number 460
Encontrado el tamaño  url_c
Fetching url for image number 461
Encontrado el tamaño  url_m
Fetching url for image number 462
Encontrado el tamaño  url_z
Fetching url for image number 463
Encontrado el tamaño  url_c
Fetching url for image number 464
Encontrado el tamaño  url_c
Fetching url for image number 465
Encontrado el tamaño  url_c
Fetching url for image number 466
Encontrado el tamaño  url_z
Fetching

Fetching url for image number 601
Encontrado el tamaño  url_c
Fetching url for image number 602
Encontrado el tamaño  url_n
Fetching url for image number 603
Encontrado el tamaño  url_c
Fetching url for image number 604
Encontrado el tamaño  url_c
Fetching url for image number 605
Encontrado el tamaño  url_c
Fetching url for image number 606
Encontrado el tamaño  url_c
Fetching url for image number 607
Encontrado el tamaño  url_z
Fetching url for image number 608
Encontrado el tamaño  url_c
Fetching url for image number 609
Encontrado el tamaño  url_c
Fetching url for image number 610
Encontrado el tamaño  url_c
Fetching url for image number 611
Encontrado el tamaño  url_c
Fetching url for image number 612
Encontrado el tamaño  url_c
Fetching url for image number 613
Encontrado el tamaño  url_c
Fetching url for image number 614
Encontrado el tamaño  url_c
Fetching url for image number 615
Encontrado el tamaño  url_c
Fetching url for image number 616
Encontrado el tamaño  url_c
Fetching

Fetching url for image number 751
Encontrado el tamaño  url_c
Fetching url for image number 752
Encontrado el tamaño  url_c
Fetching url for image number 753
Encontrado el tamaño  url_c
Fetching url for image number 754
Encontrado el tamaño  url_c
Fetching url for image number 755
Encontrado el tamaño  url_c
Fetching url for image number 756
Encontrado el tamaño  url_c
Fetching url for image number 757
Encontrado el tamaño  url_c
Fetching url for image number 758
Encontrado el tamaño  url_z
Fetching url for image number 759
Encontrado el tamaño  url_c
Fetching url for image number 760
Encontrado el tamaño  url_c
Fetching url for image number 761
Encontrado el tamaño  url_c
Fetching url for image number 762
Encontrado el tamaño  url_c
Fetching url for image number 763
Encontrado el tamaño  url_c
Fetching url for image number 764
Encontrado el tamaño  url_z
Fetching url for image number 765
Encontrado el tamaño  url_c
Fetching url for image number 766
Encontrado el tamaño  url_c
Fetching

Fetching url for image number 901
Encontrado el tamaño  url_c
Fetching url for image number 902
Encontrado el tamaño  url_c
Fetching url for image number 903
Encontrado el tamaño  url_c
Fetching url for image number 904
Encontrado el tamaño  url_c
Fetching url for image number 905
Encontrado el tamaño  url_c
Fetching url for image number 906
Encontrado el tamaño  url_c
Fetching url for image number 907
Encontrado el tamaño  url_z
Fetching url for image number 908
Encontrado el tamaño  url_c
Fetching url for image number 909
Encontrado el tamaño  url_c
Fetching url for image number 910
Encontrado el tamaño  url_z
Fetching url for image number 911
Encontrado el tamaño  url_c
Fetching url for image number 912
Encontrado el tamaño  url_c
Fetching url for image number 913
Encontrado el tamaño  url_c
Fetching url for image number 914
Encontrado el tamaño  url_c
Fetching url for image number 915
Encontrado el tamaño  url_c
Fetching url for image number 916
Encontrado el tamaño  url_c
Fetching

In [51]:
urls.head()

0    https://live.staticflickr.com/65535/5049753075...
1    https://live.staticflickr.com/8286/29091917084...
2    https://live.staticflickr.com/8108/29093761373...
3    https://live.staticflickr.com/65535/4997678898...
4    https://live.staticflickr.com/3009/3092675933_...
dtype: object

In [52]:
urls[1]

'https://live.staticflickr.com/8286/29091917084_fdbaa0a421_c.jpg'

In [46]:
urls[1].split("/")[-1]

'29091917084_fdbaa0a421_c.jpg'

In [53]:
!ls -l

total 1280
-rw-rw-r-- 1 dsc dsc  54346 feb 24 22:56  cats_and_dogs.ipynb
-rw-rw-r-- 1 dsc dsc  28300 mar  6 23:24 'causa limeña_urls.csv'
-rw-rw-r-- 1 dsc dsc  61537 mar  6 23:43  chaufa_urls.csv
-rw-rw-r-- 1 dsc dsc   1302 mar  6 18:23  croquetas.csv
-rw-rw-r-- 1 dsc dsc  33770 mar 11 23:11  croquetas_urls.csv
-rw-rw-r-- 1 dsc dsc  50321 mar  7 20:53 'ensaladilla rusa_urls.csv'
-rw-rw-r-- 1 dsc dsc 169815 mar 10 23:02  fabada_urls.csv
-rw-rw-r-- 1 dsc dsc 170229 mar 11 23:28  flickr_scrap.ipynb
-rw-rw-r-- 1 dsc dsc  58995 mar  7 21:06 'gambas al ajillo_urls.csv'
-rw-rw-r-- 1 dsc dsc  66907 mar  7 00:13  gazpacho_urls.csv
-rw-rw-r-- 1 dsc dsc  67093 mar  6 22:27  huancaina_urls.csv
-rw-rw-r-- 1 dsc dsc  62979 mar  7 23:02 'huevos rotos_urls.csv'
-rw-rw-r-- 1 dsc dsc  66735 mar  6 22:06  paella_urls.csv
-rw-rw-r-- 1 dsc dsc  60636 mar  7 00:25  patatasbravas_urls.csv
-rw-rw-r-- 1 dsc dsc  67079 mar 11 23:29 'pulpo a la gallega_urls.csv'
-rw-rw-r-- 1 dsc dsc     22 feb 21 19:48  README.m

In [9]:
import requests
import os

def download_images(file_name, image_tag):
    urls = []
    doc = pd.read_csv(file_name, header=None)
    
    for row in doc.iterrows():
        if row[1][1].startswith('https'):
            #print('yuhuuuu')
            urls.append(row[1][1])
    
    if not os.path.isdir(os.path.join('/home/dsc/tfm_data/',image_tag.split("_")[0])):
        os.mkdir(os.path.join('/home/dsc/tfm_data/', image_tag.split("_")[0]))
    
    
    for url in enumerate(urls):
        resp=requests.get(url[1], stream=True)
        path_to_write = os.path.join('/home/dsc/tfm_data/',image_tag.split("_")[0],url[1].split("/")[-1])
        outfile=open(path_to_write,'wb')
        outfile.write(resp.content)
        outfile.close()
        print("Done downloading {} of {}".format(url[0]+1,len(urls)))

In [37]:
import requests
import os

def download_images(file_name, image_tag, ruta_destino):
    urls = []
    doc = pd.read_csv(file_name, header=None)
    
    for row in doc.iterrows():
        if row[1][1].startswith('https'):
            #print('yuhuuuu')
            urls.append(row[1][1])
    
    if not os.path.isdir(os.path.join(ruta_destino, image_tag.split("_")[0])):
        os.mkdir(os.path.join(ruta_destino, image_tag.split("_")[0]))
    
    
    for url in enumerate(urls):
        resp=requests.get(url[1], stream=True)
        path_to_write = os.path.join(ruta_destino, image_tag.split("_")[0],url[1].split("/")[-1])
        outfile=open(path_to_write,'wb')
        outfile.write(resp.content)
        outfile.close()
        print("Done downloading {} of {}".format(url[0]+1,len(urls)))

In [79]:
download_images(file_name = 'pulpo a la gallega_urls.csv', image_tag = 'pulpo a la gallega', 
               ruta_destino = '/home/dsc/tfm_data/')

Done downloading 1 of 100
Done downloading 2 of 100
Done downloading 3 of 100
Done downloading 4 of 100
Done downloading 5 of 100
Done downloading 6 of 100
Done downloading 7 of 100
Done downloading 8 of 100
Done downloading 9 of 100
Done downloading 10 of 100
Done downloading 11 of 100
Done downloading 12 of 100
Done downloading 13 of 100
Done downloading 14 of 100
Done downloading 15 of 100
Done downloading 16 of 100
Done downloading 17 of 100
Done downloading 18 of 100
Done downloading 19 of 100
Done downloading 20 of 100
Done downloading 21 of 100
Done downloading 22 of 100
Done downloading 23 of 100
Done downloading 24 of 100
Done downloading 25 of 100
Done downloading 26 of 100
Done downloading 27 of 100
Done downloading 28 of 100
Done downloading 29 of 100
Done downloading 30 of 100
Done downloading 31 of 100
Done downloading 32 of 100
Done downloading 33 of 100
Done downloading 34 of 100
Done downloading 35 of 100
Done downloading 36 of 100
Done downloading 37 of 100
Done downl

Vamos a trabajar en esta parte del documento, cómo podemos recoger urls y ver si las tenemos (esas fotos) guardas en disco, si están, continuaremos buscando en flickr. Si no están, las guardamos y seguimos hasta llegar al máximo del contador: (para ello ejecutamos get_urls() y probamos con sus salidas)

In [55]:
urls[1]

'https://live.staticflickr.com/8286/29091917084_fdbaa0a421_c.jpg'

In [56]:
urls[1].split("/")[-1] # este va a ser el nombre definitivo de la foto. 

'29091917084_fdbaa0a421_c.jpg'

In [16]:
import requests
import os

In [80]:
for url in enumerate(urls):
    if os.path.isfile(os.path.join('/home/dsc/tfm_data/pulpo a la gallega', url[1].split("/")[-1])):
        print('El archivo existe.');
    else:
        print('El archivo no existe')

El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo e

In [21]:
os.path.join('/home/dsc/tfm_data/', urls[1].split("/")[-1])

'/home/dsc/tfm_data/3470701849_320ec726e2_c.jpg'

In [86]:
def get_urls(image_tag, max):
    
    photos = get_photos(image_tag)
    counter=0
    urls=[]

    for photo in photos:
        if counter < max:
            #counter += 1
            #print("Fetching url for image number {}".format(counter))
            try:
                url = get_url(photo)  # get preffered size url
                
                if os.path.isfile(os.path.join('/home/dsc/tfm_data/pulpo a la gallega', url[1].split("/")[-1])):
                    print('El archivo existe.')
                    #continue
                else:
                    print('El archivo no existe')
                    urls.append(url)
                    counter += 1
            except:    
                 print("Url for image number {} could not be fetched".format(count))
            # if no url for the desired sizes then try with the next photo
        else:
            print("Done fetching urls, fetched {} urls out of {}".format(len(urls), max))
            break
    print("Writing out the urls in the current directory")
    urls = pd.Series(urls)
    urls.to_csv(image_tag + '_urls.csv')
    print('Listo!')
    

    return urls

In [87]:
prueba = get_urls('pulpo a la gallega', 1)

Encontrado el tamaño  url_c
El archivo no existe
Done fetching urls, fetched 1 urls out of 1
Writing out the urls in the current directory
Listo!


In [89]:
prueba[0]

'https://live.staticflickr.com/65535/50497530757_3f0f9e41db_c.jpg'